In [ ]:
import os
import numpy as np
import h5py
import matplotlib.pyplot as plt
from IPython.display import Image

# Workflow for each state point

This file mimics the process for manually managing a computational workflow in a nested directory structure stored in `random_walk_project`.

Each heading in this notebook is roughly one step ("operation"). 

simulate --> compute_squared_displacements --> compute_mean_squared_displacement --> plot_msd

## Set parameters for this run


In [ ]:
run_steps = 5000

mean = 0
standard_deviation = 0.775
replica = 0
seed = 221141651

# get file path
path = os.path.join("random_walk_project",
                     "standard_deviation", str(standard_deviation),
                     f"replica_{replica}_seed_{seed}"
                    )


## Run simulation

In [ ]:
generator = np.random.default_rng(seed)

# Generate all moves at once since they are independent
moves = generator.normal(mean, standard_deviation, (run_steps, 2))
positions = np.zeros((run_steps + 1, 2), dtype=float)

# Store the results: perform a cumulative sum of all moves starting from the origin
positions[1:] = np.cumsum(moves, axis=0)

with h5py.File(os.path.join(path, "signac_data.h5"), 'w') as hf:
    hf.create_dataset("positions", data=positions, dtype='f')

## Compute squared displacements

In [ ]:
squared_displacement = np.sum(positions * positions, axis=1)

with h5py.File(os.path.join(path, "signac_data.h5"), 'w') as hf:
    hf.create_dataset("squared_displacement", data=squared_displacement, dtype='f')


## Compute MSD

In [ ]:
# pull data from replicates
root = os.path.dirname(path)
replica_labels = [f for f in os.listdir(root) if not f.startswith('.')]
replica_labels = sorted(replica_labels)
print(replica_labels)

# compute msd
msd = np.zeros(run_steps + 1, dtype='float64')
for rep in replica_labels:
    rep_hf = h5py.File(os.path.join(root, rep, "signac_data.h5"), 'r')
    msd += rep_hf.get("squared_displacement")
    rep_hf.close()
msd /= len(replica_labels)


with h5py.File(os.path.join(path, "signac_data.h5"), 'w') as hf:
    hf.create_dataset("msd", data=msd, dtype='f')


## Plot MSD

In [ ]:
fig, ax = plt.subplots()
ax.plot(msd)
ax.set_title(f"MSD for standard deviation {standard_deviation}")
ax.set_xlabel("x")
ax.set_ylabel("MSD")
# Only save figure to the first replica
fig.savefig(os.path.join(path,"msd.png"))
plt.close()
Image(os.path.join(path,"msd.png"))

extras
- Compute radius of gyration
- compute end to end distance
- plot histogram